In [103]:
import pandas as pd
import numpy as np
import math
from scipy.io import savemat

def llh2local(llh, origin):
    a = 6378137.0
    e = 0.08209443794970
    llh = np.radians(llh)
    origin = np.radians(origin)
    xy = np.zeros((2, llh.shape[1]))
    z = llh[1, :] != 0
    dlambda = llh[0, z] - origin[0]
    M = a * ((1 - e**2 / 4 - 3 * e**4 / 64 - 5 * e**6 / 256) * llh[1, z] -
             (3 * e**2 / 8 + 3 * e**4 / 32 + 45 * e**6 / 1024) * np.sin(2 * llh[1, z]) +
             (15 * e**4 / 256 + 45 * e**6 / 1024) * np.sin(4 * llh[1, z]) -
             (35 * e**6 / 3072) * np.sin(6 * llh[1, z]))
    M0 = a * ((1 - e**2 / 4 - 3 * e**4 / 64 - 5 * e**6 / 256) * origin[1] -
              (3 * e**2 / 8 + 3 * e**4 / 32 + 45 * e**6 / 1024) * np.sin(2 * origin[1]) +
              (15 * e**4 / 256 + 45 * e**6 / 1024) * np.sin(4 * origin[1]) -
              (35 * e**6 / 3072) * np.sin(6 * origin[1]))
    N = a / np.sqrt(1 - e**2 * np.sin(llh[1, z])**2)
    E = dlambda * np.sin(llh[1, z])
    
    def cot(x):
        return 1 / np.tan(x)
    
    xy[0, z] = N * cot(llh[1, z]) * np.sin(E)
    xy[1, z] = M - M0 + N * cot(llh[1, z]) * (1 - np.cos(E))
    dlambda = llh[0, ~z] - origin[0]
    xy[0, ~z] = a * dlambda
    xy[1, ~z] = -M0
    xy = xy / 1000
    return xy

def calculate_fault_endpoints(center_lat, center_lon, strike, length):
    strike_rad = math.radians(strike)
    R = 6371.0
    d_lat = (length / 2) * math.cos(strike_rad) / R
    d_lon = (length / 2) * math.sin(strike_rad) / (R * math.cos(math.radians(center_lat)))
    d_lat = math.degrees(d_lat)
    d_lon = math.degrees(d_lon)
    start_lat = center_lat - d_lat
    start_lon = center_lon - d_lon
    end_lat = center_lat + d_lat
    end_lon = center_lon + d_lon

    return start_lat, start_lon, end_lat, end_lon

def strike2inc(strike):
    deg2rad = np.pi / 180
    if (strike >= 0) and (strike < 90):
        str_rad = strike * deg2rad
        xinc = np.sin(str_rad)
        yinc = np.cos(str_rad)
    elif (strike >= 90) and (strike < 180):
        str_rad = (strike - 90) * deg2rad
        xinc = np.cos(str_rad)
        yinc = -np.sin(str_rad)
    elif (strike >= 180) and (strike < 270):
        str_rad = (strike - 180) * deg2rad
        xinc = -np.sin(str_rad)
        yinc = -np.cos(str_rad)
    elif (strike >= 270) and (strike < 360):
        str_rad = (strike - 270) * deg2rad
        xinc = -np.cos(str_rad)
        yinc = np.sin(str_rad)
    return xinc, yinc

def generate_points_along_strike(start_lat, start_lon, end_lat, end_lon, size_strike, strike, origin):
    start_xy = llh2local(np.array([[start_lon, start_lat]]).T, origin)
    end_xy = llh2local(np.array([[end_lon, end_lat]]).T, origin)
    # print(start_xy)
    delta_x = end_xy[0] - start_xy[0]
    delta_y = end_xy[1] - start_xy[1]
    total_distance_km = np.sqrt(delta_x**2 + delta_y**2)[0]
    num_points = max(int(total_distance_km / size_strike), 2)
    adjusted_size_strike = total_distance_km / (num_points-1)
    # print(num_points, adjusted_size_strike,(num_points-1)*adjusted_size_strike, total_distance_km)
    xinc, yinc = strike2inc(strike)
    points = []
    # print((num_points-1)*adjusted_size_strike)
    for i in range(num_points + 1):
        x = start_xy[0] + i * xinc * adjusted_size_strike
        y = start_xy[1] + i * yinc * adjusted_size_strike
        points.append((x[0], y[0]))
    # Calculate the center of each patch
    patch_centers = []
    for i in range(num_points):
        center_x = (points[i][0] + points[i + 1][0]) / 2
        center_y = (points[i][1] + points[i + 1][1]) / 2
        patch_centers.append((center_x, center_y))

    return patch_centers, points, adjusted_size_strike
    

def generate_points_along_dip(points, strike, dip, cum_depth,lenght):
    dip_rad = np.radians(dip)  # Convert dip to radian
    strike_rad=np.radians((360-strike))
    depth = cum_depth
    # print("Cumulative depth:", cum_depth)
    
    delta_x =1*depth *np.cos(dip_rad)*np.sin(strike_rad)
    delta_y =1*depth *np.cos(dip_rad)*np.cos(strike_rad)
 
    
    # print(f'delta_x: {delta_x}, delta_y: {delta_y}')
    
    dip_points = []
    for point in points:
        new_x = point[0] + delta_x
        new_y = point[1] + delta_y
        dip_points.append((new_x, new_y))

    
    return dip_points

def calculate_increasing_depths(size_patches, depth, inc_factor=1.2):
    depths = []
    current_depth = size_patches
    cumulative_sum = 0
    while cumulative_sum + current_depth < depth:
        rounded_depth = round(current_depth * 2) / 2
        depths.append(rounded_depth)
        cumulative_sum += rounded_depth
        current_depth *= inc_factor
    
    remainder = depth - cumulative_sum
    if remainder > 0:
        rounded_remainder = round(remainder * 2) / 2
        depths.append(rounded_remainder)
    
    return depths

def process_fault_data(fault_inp, origin):
    all_patches = []

    for index, row in fault_inp.iterrows():
        center_lat = row['lat(deg)']
        center_lon = row['lon(deg)']
        dip = row['dip(deg)']
        strike = row['strike(deg)']
        size_patches = row['size_patches(km)']
        length = row['length(km)']
        inc_factor = row['increasing_factor(scalar)']

        start_lat, start_lon, end_lat, end_lon = calculate_fault_endpoints(center_lat, center_lon, strike, length)
        depth = row['depth(km)']
        inc_depth = calculate_increasing_depths(size_patches, depth, inc_factor)
        cumulative_depth = 0

        # Calculate cumulative depths excluding the last value of inc_depth for the calcualte the points along the depth!
        cum_depth_values = np.cumsum(inc_depth[:-1])  / np.sin(np.radians(-1 * dip))
        cum_depth_iter = iter(cum_depth_values)  # Create an iterator for cumulative depths
        #print(f'cum iteration {cum_depth_values}')
        ###Points calculation the top surface!
        for variable_depth in inc_depth:
            if variable_depth==inc_depth[0]:# or variable_depth==inc_depth[1] :
                #print(f'first estimated size for top patches is {variable_depth}')
                points_center, points= generate_points_along_strike(start_lat, start_lon, end_lat, end_lon, variable_depth, strike, origin)
                for i in range(len(points_center) - 1):
                    # print(f'points: {points[i]}')
                    # print(f'points_center: {points_center[i]}')
                    point1_local = points_center[i]
                    point2_local = points_center[i + 1]
                    length = np.sqrt((point2_local[0] - point1_local[0])**2 + (point2_local[1] - point1_local[1])**2)
                    all_patches.append({
                        'Length': length,
                        'Width': variable_depth/np.sin(np.radians(-1*dip)),
                        'Z_center': cumulative_depth,
                        'Dip': dip,
                        'Strike': strike,
                        'X_center': point1_local[0],
                        'Y_center': point1_local[1],
                        'Field_8': 0,
                        'Field_9': 0,
                        'Field_10': 0
                    })
                cumulative_depth += variable_depth #/np.sin(np.radians(-1*dip))
                
#Points calculations along the depth depending the top coordinates and dip angles! 
#If the dip is 89.99 there is no problem. My lord why everytime I struggle such a kind of example? I would interest much simpler fault model
            
            else:
                try:
                    cum_depth = next(cum_depth_iter)
                    # print(f'Next patches size along the depth: {variable_depth}, distance_from_top: {cum_depth}')
                except StopIteration:
                    print('No more cumulative depth values available.')
                #print(f'cumulative depth {cum_depth}')
                points_center, points = generate_points_along_strike(start_lat, start_lon, end_lat, end_lon, variable_depth, strike, origin)
                points_dip = generate_points_along_dip(points_center, strike, dip, cum_depth)
                for i in range(len(points_dip) - 1):
                    point1_local = points_dip[i]
                    point2_local = points_dip[i + 1]
                    length = np.sqrt((point2_local[0] - point1_local[0])**2 + (point2_local[1] - point1_local[1])**2)
                    all_patches.append({
                        'Length': length,
                        'Width': variable_depth / np.sin(np.radians(-1 * dip)),
                        'Z_center': cumulative_depth,
                        'Dip': dip,
                        'Strike': strike,
                        'X_center': point1_local[0],
                        'Y_center': point1_local[1],
                        'Field_8': 0,
                        'Field_9': 0,
                        'Field_10': 0
                    })
                cumulative_depth += variable_depth #/ np.sin(np.radians(-1 * dip))
                # print(f'other_cum {cumulative_depth}')

    return all_patches

def main(file_path, origin):
    fault_inp = pd.read_csv(file_path, index_col=False)
    all_patches = process_fault_data(fault_inp, origin)
    patches_df = pd.DataFrame(all_patches)
    # Transpose the DataFrame to get the desired format
    patches_array = patches_df.T.to_numpy()
    # Convert DataFrame to .mat file
    savemat('fault_patches1.mat', {'fault_patches': patches_array})
    return patches_df

# Run the main function with the appropriate file path and origin
file_path = 'fault_center.txt'
origin = [37.5, 37.5]  # Define your origin
patches_df = main(file_path, origin)
patches_df

,Length,Width,Z_center,Dip,Strike,X_center,Y_center,Field_8,Field_9,Field_10
0,35.995449,23.094011,0.0,-60.0,220.0,-61.164926,-23.516694,0,0,0
1,2.249716,2.309401,0.0,-60.0,220.0,-50.319256,-10.591328,0,0,0
2,2.249716,2.309401,0.0,-60.0,220.0,-51.765346,-12.314711,0,0,0
3,2.249716,2.309401,0.0,-60.0,220.0,-53.211435,-14.038093,0,0,0
4,2.249716,2.309401,0.0,-60.0,220.0,-54.657524,-15.761475,0,0,0
...,...,...,...,...,...,...,...,...,...,...
57,5.999242,5.773503,15.0,-60.0,220.0,-49.813865,-23.257305,0,0,0
58,5.999242,5.773503,15.0,-60.0,220.0,-53.670103,-27.852991,0,0,0
59,5.999242,5.773503,15.0,-60.0,220.0,-57.526341,-32.448677,0,0,0
60,5.999242,5.773503,15.0,-60.0,220.0,-61.382579,-37.044362,0,0,0
